In [ ]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

In [ ]:
scyjava.config.add_option('-Xmx30g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
## Something about this init() function changes the current working directory.
ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

In [ ]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
ov = Overlay()

In [ ]:
Overlay = scyjava.jimport('ij.gui.Overlay')
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

In [ ]:
directory_path = '/home/saka/Documents/Lab stuff/confocal/exp2'
#directory_path = '/home/saka/Documents/Zhihui/timelapse/images'
os.chdir(directory_path)

In [ ]:
root = tk.Tk()
root.withdraw()
filepath = filedialog.askopenfilename()

In [ ]:
image = ij.io().open(filepath)
#ij.ui().show(image)
imp = ij.py.to_imageplus(image)

In [ ]:
ij.ui().show(imp)

In [ ]:


#wanted_channel = 2
#blue_channel = image[:, :, wanted_channel,:,:]
#imp = ij.py.to_imageplus(blue_channel)



In [ ]:
wanted_channel = 3
channel = image[:, :, wanted_channel, :, :]
imp = ij.py.to_imageplus(channel)
imp.setDimensions(1, 11, 121)
#ij.ui().show(imp)
#ZProjector = scyjava.jimport("ij.plugin.ZProjector")()
#method = 'sum all'
#z_projector_result = ZProjector.run(imp, method)
#z_collapsed_image = ij.py.from_java(z_projector_result)
#z_collapsed_dataset = ij.py.to_dataset(z_collapsed_image)
#ij.ui().show(z_collapsed_dataset)

In [ ]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['Z']

In [ ]:
for i in range (data_info['T']):
    macro = f"Stack.setFrame({i + 1})"
    ij.py.run_macro(macro)
    input_txt = Path(f"{directory_path}/frame_{i}_cp_outlines.txt")
        for timepoint in range(1, num_frame):
        frame_number = timepoint - 1
        input_txt = Path(f"{directory_path}/outputs/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt\")

In [ ]:
file_prefix = "Image_T"
for timepoint in range(1, num_frame + 1):
    frame_number = f"{timepoint:04d}"
    #input_txt = Path(f"{directory_path}/txt_outlines/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    input_txt = Path(f"{directory_path}/txt_outlines/{file_prefix}{frame_number}_Overlay_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setZ(timepoint)   
            rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

In [ ]:
wanted_channel = 3
channel = image[:, :, wanted_channel, :, :]
imp = ij.py.to_imageplus(channel)
imp.setDimensions(1, 11, 121)
ij.ui().show(imp)

In [ ]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['T'] + 1

In [ ]:
for timepoint in range(1, num_frame):
    frame_number = timepoint - 1
    input_txt = Path(f"{directory_path}/outputs/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setT(timepoint)   
            rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

In [ ]:
file_path = Path(f"{directory_path}/results_bacteria.csv")
df = pandas.read_csv(file_path)
#df = pandas.read_csv(dataset)


In [ ]:
selected_column = df.iloc[:, 1]
new_column_name = "ROI_ID"
df = df.rename(columns = selected_column : new_column_name)

In [ ]:
selected_column = df.iloc[:, 1]
selected_column.rename(new_column_name)

In [ ]:
df.head()

In [ ]:
x_column='X'
y_column='Y'
z_column='Slice'
gdf = geopandas.GeoDataFrame(
        df, geometry=geopandas.points_from_xy(df[x_column], df[y_column], df[z_column]))

In [ ]:
gdf.head()

In [ ]:
final_time = gdf.Frame.max()

In [ ]:
pairwise_distances = []
    for start_time in range(0, final_time):
        i = start_time
        j = i + 1
        ti_idx = gdf.Frame == i
        tj_idx = gdf.Frame == j

In [ ]:
ij.ui().show(imp)

In [ ]:
roi = rm.select(4)
duplicator_result = Duplicator.run(imp, 1, 1, 1, 11, 1, 1)

In [ ]:
ij.ui().show(duplicator_result)

In [ ]:
cell_id = nearest.keys()
for i in cell_id:
#this array can be used with the "nearest" dataset. This one is just an example
    celli = nearest[i]
    for idx in celli
        roi = rm.select(idx)
        Stack.getPosition(channel, slice, frame);
    	run("Duplicate...", "duplicate frames="+frame);
        selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");

In [ ]:
prout = [17,74,132,185,240,295,347,397,440,488,536,586,645,
697,750,804,856,645,697,750,804,856,909,958,1008,1059,1111,
1160,1210,1260,1307,1352,1395,1441,1486,1533,1580,1628,1670,
1716,1756,1800,1849,1897,1942,1986,2027,2072,2119,2164,2205,
2248,2286,2334,2375,2412,2456,2498,2538,2576,2613,2652]

In [ ]:
for i in prout:
    roi = rm.select(i)
    